In [5]:
import pandas as pd
import json
import os
import regex as re

In [6]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.3.0

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-60afe087-690b-4ff2-94e6-50f3a59c9a6f;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 251ms :: artifacts dl 13ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

23/11/10 19:09:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/10 19:09:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
3.3.0


In [8]:
%%time
# Read in data from project bucket
bucket = "project17-bucket-alex"
#output_prefix_data = "project_2022"

# List of 12 directories each containing 1 month of data
directories = ["project_2022_"+str(i)+"/submissions" for i in range(1,13)]

# Iterate through 12 directories and merge each monthly data set to create one big data set
df_subs = None
for directory in directories:
    s3_path = f"s3a://{bucket}/{directory}"
    month_df = spark.read.parquet(s3_path, header = True)
    
    if df_subs is None:
        df_subs = month_df
    else:
        df_subs = df_subs.union(month_df)

23/11/10 19:09:42 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


23/11/10 19:09:47 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
CPU times: user 16.8 ms, sys: 8.29 ms, total: 25.1 ms
Wall time: 12.5 s


In [9]:
%%time
# Read in data from project bucket
bucket = "project17-bucket-alex"
#output_prefix_data = "project_2022"

# List of 12 directories each containing 1 month of data
directories = ["project_2022_"+str(i)+"/comments" for i in range(1,13)]

# Iterate through 12 directories and merge each monthly data set to create one big data set
df_comms = None
for directory in directories:
    s3_path = f"s3a://{bucket}/{directory}"
    month_df = spark.read.parquet(s3_path, header = True)
    
    if df_comms is None:
        df_comms = month_df
    else:
        df_comms = df_comms.union(month_df)

CPU times: user 12.1 ms, sys: 3.76 ms, total: 15.8 ms
Wall time: 6.01 s


In [10]:
df_comms.select(['subreddit', 'author', 'body', 'parent_id', 'link_id', 'id', 'created_utc', 'controversiality']).show(10)

+-----------------+--------------------+--------------------+----------+---------+-------+-------------------+----------------+
|        subreddit|              author|                body| parent_id|  link_id|     id|        created_utc|controversiality|
+-----------------+--------------------+--------------------+----------+---------+-------+-------------------+----------------+
|    AmItheAsshole|         beckydragon|                 NTA| t3_rz9uu3|t3_rz9uu3|hs0rusg|2022-01-10 04:49:57|               0|
|    AmItheAsshole|        Cactus_chuck|NTA. My partners ...| t3_s0baev|t3_s0baev|hs0rusr|2022-01-10 04:49:57|               0|
|    AmItheAsshole|   Red-belliedOrator|INFO\n\nIn genera...| t3_s0a5hn|t3_s0a5hn|hs0rut9|2022-01-10 04:49:57|               0|
|NoStupidQuestions|  SoMuchForLongevity|You couldn't heat...| t3_s0b5be|t3_s0b5be|hs0rutc|2022-01-10 04:49:57|               0|
|NoStupidQuestions|          MMmason651|it wouldn't taste...| t3_s0axsd|t3_s0axsd|hs0rutm|2022-01-10 04:

In [11]:
# Select first 5 rows of 'body' column
first_50_bodies = df_comms.select("body").limit(50).collect()

# Convert to a list of strings
bodies_list = [row.body for row in first_50_bodies]

In [12]:
!pip install transformers
!pip install torch


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [13]:
import transformers
import torch
print(torch.__version__)

2.1.0+cu121


In [14]:
from transformers import pipeline

# Initialize the summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Apply summarization to each body text
summaries = [summarizer(body, max_length=130, min_length=30, do_sample=False)[0]['summary_text'] for body in bodies_list]

Your max_length is set to 130, but your input_length is only 4. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)
Your max_length is set to 130, but your input_length is only 45. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)
Your max_length is set to 130, but your input_length is only 30. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Your max_length is set to 130, but your input_length is only 7. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)
Your max

In [15]:
for body in bodies_list[0:10]:
    print()
    print(body)


NTA

NTA. My partners birthday is within days of christmas as well. I usually do one large gift on one day and then just a small gift on the other so that she still gets a gift to open.

INFO

In general, I think that when a wife agrees to stay with a husband, or vice versa, eventually, the family has to respect her decision. In this case, there is another family member involved, the cousin, but I'm not quite clear on why she is so upset. Did your uncle try to coerce her into a relationship? Is he delusional or fixated on her? Why would the uncle tell you that he was leaving his wife for your cousin, if the cousin hadn't agreed to this? I guess I'm just really unclear on what the cousin's role is here, and why she's so upset about it.

You couldn't heat it even one degree by stirring it vigorously by hand.

With powerful machinery, though, you could get somewhere.

it wouldn't taste good

Play with my boobs

This happened to me too. Also in third grade, our teacher told us we were doi

In [16]:
for summary in summaries[0:10]:
    print()
    print(summary)


CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of NTA for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots.

My partners birthday is within days of christmas as well. I usually do one large gift on one day and then just a small gift on the other so that she still gets a gift to open.

In general, I think that when a wife agrees to stay with a husband, or vice versa, eventually, the family has to respect her decision. In this case, there is another family member involved, the cousin, but I'm not quite clear on why she is so upset. Did your uncle try to coerce her into a relationship? Is he delusional or fixated on her?

With powerful machinery, though, you could get somewhere. You couldn't heat it even one degree by stirring it vigorously by hand. But you could make a difference with powerful machinery.

CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your b

In [17]:
for body in bodies_list:
    print()
    print(body)


NTA

NTA. My partners birthday is within days of christmas as well. I usually do one large gift on one day and then just a small gift on the other so that she still gets a gift to open.

INFO

In general, I think that when a wife agrees to stay with a husband, or vice versa, eventually, the family has to respect her decision. In this case, there is another family member involved, the cousin, but I'm not quite clear on why she is so upset. Did your uncle try to coerce her into a relationship? Is he delusional or fixated on her? Why would the uncle tell you that he was leaving his wife for your cousin, if the cousin hadn't agreed to this? I guess I'm just really unclear on what the cousin's role is here, and why she's so upset about it.

You couldn't heat it even one degree by stirring it vigorously by hand.

With powerful machinery, though, you could get somewhere.

it wouldn't taste good

Play with my boobs

This happened to me too. Also in third grade, our teacher told us we were doi

In [18]:
for summary in summaries:
    print()
    print(summary)


CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of NTA for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots.

My partners birthday is within days of christmas as well. I usually do one large gift on one day and then just a small gift on the other so that she still gets a gift to open.

In general, I think that when a wife agrees to stay with a husband, or vice versa, eventually, the family has to respect her decision. In this case, there is another family member involved, the cousin, but I'm not quite clear on why she is so upset. Did your uncle try to coerce her into a relationship? Is he delusional or fixated on her?

With powerful machinery, though, you could get somewhere. You couldn't heat it even one degree by stirring it vigorously by hand. But you could make a difference with powerful machinery.

CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your b

In [20]:
type(bodies_list)

list

In [21]:
type(summaries)

list

In [22]:
with open('pre_summary.txt', 'w') as file:
    for item in bodies_list:
        file.write("%s\n" % item)

In [23]:
with open('summaries.txt', 'w') as file:
    for item in summaries:
        file.write("%s\n" % item)

In [ ]:
#code for processing data was adapted with assistance from ChatGPT